# Wrong

In [52]:
# import xml.etree.ElementTree as ET

# def parse_annotation(xml_path):
#     f = open(xml_path, "r")
#     xml_string = f.read()
#     root = ET.fromstring(xml_string)
    
#     annotation = {}
#     annotation['folder'] = root.find('folder').text
#     annotation['filename'] = root.find('filename').text
    
#     size = root.find('size')
#     annotation['width'] = int(size.find('width').text)
#     annotation['height'] = int(size.find('height').text)
#     try:
#         objects = root.findall('object')
#         print(objects)
#         annotation['objects'] = []
        
#         for object_element in objects:
#             obj = {}
#             obj['name'] = object_element.find('name').text
            
#             bndbox = object_element.find('bndbox')
#             obj['bndbox'] = {}
#             obj['bndbox']['xmin'] = int(bndbox.find('xmin').text)
#             obj['bndbox']['ymin'] = int(bndbox.find('ymin').text)
#             obj['bndbox']['xmax'] = int(bndbox.find('xmax').text)
#             obj['bndbox']['ymax'] = int(bndbox.find('ymax').text)
            
#             annotation['objects'].append(obj)
#     except:
#         print(xml_path)

#     # parsed_data = parse_annotation(xml_string)
#     return annotation
# # print(parsed_data)

In [ ]:
# def get_label(parsed_data):
#     try:
#         obj_name = [dict['name'] for dict in parsed_data['objects']]
#     except:
#         obj_name = []
#     return obj_name

In [ ]:
import os
import pandas as pd

code = []
for list_rdd in os.listdir("RDD2022"):
    for annot in os.listdir(f"RDD2022/{list_rdd}/train/annotations/xmls"):
        data = parse_annotation(os.path.join(f"RDD2022/{list_rdd}/train/annotations/xmls", annot))
        get_l = get_label(data)
        code.extend(get_l)

df = pd.DataFrame(code)
df[0].unique()

In [ ]:
def get_obj_code(obj_name):
    if obj_name == "D00":
        return 0
    elif obj_name == "D01":
        return 1
    elif obj_name == "D10":
        return 2
    elif obj_name == "D11":
        return 3
    elif obj_name == "D20":
        return 4
    elif obj_name == "D40":
        return 5
    elif obj_name == "D43":
        return 6
    elif obj_name == "D44":
        return 7
    elif obj_name == "D50":
        return 8 # manhole cover
    elif obj_name == "Block crack":
        return 9
    elif obj_name == "Repair":
        return 10

In [ ]:
def get_code(parsed_data):
    try:
        # obj_name = [get_obj_code(dict['name']) for dict in parsed_data['objects']]
        obj_name = [0 for dict in parsed_data['objects']]
    except:
        obj_name = []
    return obj_name

In [ ]:
# xml_path = './RDD2022/Norway/train/annotations/xmls/Norway_007037.xml'
# parsed_data = parse_annotation(xml_path)
# print(parsed_data)
# # obj_name = [0 for dict in parsed_data['objects'] if len(parsed_data['objects']) > 0]
# # print(obj_name)
# if len(parsed_data['objects']) > 0:
#     print("y")

In [ ]:
def write_labels(label_path, parsed_data):
    if len(parsed_data['objects']) > 0:
        try:
            obj_name = [0 for dict in parsed_data['objects']]
            img_width = parsed_data['width']
            img_height = parsed_data['height']
            x_center = [((dict['bndbox']['xmax'] + dict['bndbox']['xmin'])/2)/img_width for dict in parsed_data['objects']]
            y_center = [((dict['bndbox']['ymax'] + dict['bndbox']['ymin'])/2)/img_height for dict in parsed_data['objects']]
            width = [(dict['bndbox']['xmax'] - dict['bndbox']['xmin'])/img_width for dict in parsed_data['objects']]
            height = [(dict['bndbox']['ymax'] - dict['bndbox']['ymin'])/img_height for dict in parsed_data['objects']]

            file = open(label_path, "a")
            for i in range(len(parsed_data['objects'])):
                if i == 0:
                    file.write(f"{obj_name[i]} {x_center[i]} {y_center[i]} {width[i]} {height[i]}")
                else:
                    file.write(f"\n{obj_name[i]} {x_center[i]} {y_center[i]} {width[i]} {height[i]}") 
        except:
            # file = open(label_path, "a")
            pass
    else:
        pass

# Data Preparation

In [2]:
import xml.etree.ElementTree as ET

def parse_annotation(xml_path):
    f = open(xml_path, "r")
    xml_string = f.read()
    root = ET.fromstring(xml_string)
    
    annotation = {}
    annotation['folder'] = root.find('folder').text
    annotation['filename'] = root.find('filename').text
    
    size = root.find('size')
    annotation['width'] = int(size.find('width').text)
    annotation['height'] = int(size.find('height').text)
    
    objects = root.findall('object')
    # print(f"Number of objects: {len(objects)}")
    annotation['objects'] = []
        
    for object_element in objects:
        obj = {}
        obj['name'] = object_element.find('name').text
            
        bndbox = object_element.find('bndbox')
        obj['bndbox'] = {}
        obj['bndbox']['xmin'] = float(bndbox.find('xmin').text)
        obj['bndbox']['ymin'] = float(bndbox.find('ymin').text)
        obj['bndbox']['xmax'] = float(bndbox.find('xmax').text)
        obj['bndbox']['ymax'] = float(bndbox.find('ymax').text)
            
        annotation['objects'].append(obj)

    return annotation

In [3]:
import os
import pandas as pd

In [ ]:
def get_obj_code(obj_name):
    if obj_name == "D00" or obj_name == "D10" or obj_name == "D20" or obj_name == "D40":
        return 0
    else:
        return -1

In [18]:
def write_labels(label_path, parsed_data):
    if len(parsed_data['objects']) > 0:
        try:
            obj_name = []
            img_width = parsed_data['width']
            img_height = parsed_data['height']
            x_center = []
            y_center = []
            width = []
            height = []

            for dict in parsed_data['objects']:
                if dict['name'] in ['D00', 'D10', 'D20', 'D40']:
                    obj_name.append(0)
                    x_center.append(((dict['bndbox']['xmax'] + dict['bndbox']['xmin'])/2)/img_width)
                    y_center.append(((dict['bndbox']['ymax'] + dict['bndbox']['ymin'])/2)/img_height)
                    width.append((dict['bndbox']['xmax'] - dict['bndbox']['xmin'])/img_width)
                    height.append((dict['bndbox']['ymax'] - dict['bndbox']['ymin'])/img_height)

            file = open(label_path, "a")
            for i in range(len(obj_name)):
                if i == 0:
                    file.write(f"{obj_name[i]} {x_center[i]} {y_center[i]} {width[i]} {height[i]}")
                else:
                    file.write(f"\n{obj_name[i]} {x_center[i]} {y_center[i]} {width[i]} {height[i]}") 
        except:
            # file = open(label_path, "a")
            pass
    else:
        pass

In [5]:
import os
import shutil

if os.path.exists("data"):
    shutil.rmtree("data")
os.mkdir("data")
os.mkdir("data/images")
os.mkdir("data/labels")
os.mkdir("data/train")
# os.mkdir("data/test")
os.mkdir("data/val")
os.mkdir("data/train/images")
# os.mkdir("data/test/images")
os.mkdir("data/val/images")
os.mkdir("data/train/labels")
# os.mkdir("data/test/labels")
os.mkdir("data/val/labels")

In [19]:
for list_rdd in os.listdir("RDD2022"):
    print(f"{list_rdd} begin")
    for annot in os.listdir(f"RDD2022/{list_rdd}/train/annotations/xmls"):
        data = parse_annotation(os.path.join(f"RDD2022/{list_rdd}/train/annotations/xmls", annot))
        txt_path = annot.split(".")[0]
        write_labels(f"data/labels/{txt_path}.txt", data)
        # shutil.copy(src=f"RDD2022/{list_rdd}/train/images/{txt_path}.jpg", dst=f"data/images/{txt_path}.jpg")
    print(f"{list_rdd} end\n")

China_Drone begin
China_Drone end

China_MotorBike begin
China_MotorBike end

Czech begin
Czech end

India begin
India end

Japan begin
Japan end

Norway begin
Norway end

United_States begin
United_States end



In [20]:
print(len(os.listdir("./data/labels")))
print(len(os.listdir("./data/images")))

26660
38384


In [8]:
len(os.listdir("./data/labels"))
len(os.listdir("./data/images"))

26660

In [13]:
no_damage = []
data_dir = "./data/images"
labels = os.listdir("./data/labels")
paths = [i.split(".")[0] for i in labels]

for image in os.listdir(data_dir):
    if image.split(".")[0] not in paths:
        no_damage.append(image)
len(no_damage)


11724

In [15]:
for image in no_damage:
    os.remove(os.path.join(data_dir, image))

In [16]:
len(os.listdir("./data/images"))

26660

In [17]:
img_dir = "./data/images"
label_dir = "./data/labels"

img_paths = []
label_paths = []
for img in os.listdir(img_dir):
    img_paths.append(os.path.join(img_dir, img))

for label in os.listdir(label_dir):
    label_paths.append(os.path.join(label_dir, label))

data_path = pd.DataFrame()
data_path['img_path'] = img_paths
data_path['label_path'] = label_paths

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(data_path, test_size=0.2, random_state=0)
# X_train, X_val = train_test_split(X_train, test_size=0.2, random_state=0)

In [19]:
data_path.to_csv("data_path.csv")
X_train.to_csv("X_train.csv")
# X_test.to_csv("X_test.csv")
X_val.to_csv("X_val.csv")

In [20]:
for img in X_train['img_path']:
    img_p = img.split("\\")[1]
    new_img = f".data/train/images/{img_p}"
    shutil.move(src=img,dst=new_img)

for label in X_train['label_path']:
    label_p = label.split("\\")[1]
    new_label = f"./data/train/labels/{label_p}"
    shutil.move(src=label,dst=new_label)

In [21]:
for img in X_val['img_path']:
    img_p = img.split("\\")[1]
    new_img = f"./data/val/images/{img_p}"
    shutil.move(src=img,dst=new_img)

for label in X_val['label_path']:
    label_p = label.split("\\")[1]
    new_label = f"./data/val/labels/{label_p}"
    shutil.move(src=label,dst=new_label)

In [22]:
# for img in X_test['img_path']:
#     img_p = img.split("\\")[1]
#     new_img = f"./data/test/images/{img_p}"
#     shutil.move(src=img,dst=new_img)

# for label in X_test['label_path']:
#     label_p = label.split("\\")[1]
#     new_label = f"./data/test/labels/{label_p}"
#     shutil.move(src=label,dst=new_label)

In [43]:
import cv2
import matplotlib.pyplot as plt

# Load the image
image_path = './data/train/images/Norway_007650.jpg'
image = cv2.imread(image_path)

# Read the annotation file
annotation_file = './data/train/labels/Norway_007650.txt'
with open(annotation_file, 'r') as file:
    lines = file.readlines()

# Draw bounding boxes
height, width, _ = image.shape
for line in lines:
    class_label, x_norm, y_norm, width_norm, height_norm = line.split()
    x_norm, y_norm, width_norm, height_norm = map(float, (x_norm, y_norm, width_norm, height_norm))

    # Convert normalized coordinates to absolute coordinates
    x = int(x_norm * width)
    y = int(y_norm * height)
    box_width = int(width_norm * width)
    box_height = int(height_norm * height)

    # Calculate the top-left and bottom-right coordinates of the bounding box
    x1 = x - box_width // 2
    y1 = y - box_height // 2
    x2 = x1 + box_width
    y2 = y1 + box_height

    # Draw the bounding box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, class_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image
# plt.imshow(image)
cv2.imshow('Annotated Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Model YOLO

In [23]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.112  Python-3.9.13 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
Setup complete  (12 CPUs, 15.4 GB RAM, 349.1/475.3 GB disk)


In [24]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
model.train(data='data.yaml', save=True, epochs=3, imgsz=640, device=0)

New https://pypi.org/project/ultralytics/8.0.120 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.112  Python-3.9.13 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=